In [ ]:
# ==============================================================================
# Autor(es): Juan Felipe Agudelo Rios                                         |
# Titulo: Model                                                               |
# Fecha creación: 23/12/2023                                                  |
# Fecha última modificación: 26/12/2023                                       |
# ==============================================================================

In [16]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from sklearn.metrics import make_scorer, fbeta_score
from xgboost import XGBClassifier  # Import XGBoost
import shap
import pandas as pd 
import os 
import regex as re

original_directory = "/Users/j.agudelo/Desktop/CasoFi/"

os.chdir(original_directory)

data=pd.read_excel("Output/DataFrames/CleanData.xlsx")

data

,Unnamed: 0,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,...,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Education_Level_num,Income_Category_num,Card_Category_num,Marital_Status_num,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Attrition_Num,Male
0,0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,...,0.000093,1,2,0,1,0,1,0,0,1
1,1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,...,0.000057,2,0,0,2,0,0,1,0,0
2,2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,...,0.000021,2,3,0,1,0,1,0,0,1
3,3,769911858,Existing Customer,40,F,4,High School,Single,Less than $40K,Blue,...,0.000134,1,0,0,2,0,0,1,0,0
4,4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,...,0.000022,0,2,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,10122,772366833,Existing Customer,50,M,2,Graduate,Single,$40K - $60K,Blue,...,0.000191,2,1,0,2,0,0,1,0,1
10123,10123,710638233,Attrited Customer,41,M,2,High School,Divorced,$40K - $60K,Blue,...,0.995270,1,1,0,0,1,0,0,1,1
10124,10124,716506083,Attrited Customer,44,F,1,High School,Married,Less than $40K,Blue,...,0.997880,1,0,0,1,0,1,0,1,0
10125,10125,717406983,Attrited Customer,30,M,2,Graduate,Married,$40K - $60K,Blue,...,0.996710,2,1,0,1,0,1,0,1,1


In [17]:

Dependent_var="attrition_num"

selected_feature_names=['customer_age',
                        'dependent_count',
                        'total_relationship_count',
                        'months_inactive_12_mon',
                        'contacts_count_12_mon',
                        'total_amt_chng_q4_q1',
                        'total_trans_ct',
                        'total_ct_chng_q4_q1',
                        'avg_utilization_ratio',
                        'education_level_num',
                        'income_category_num',
                        'card_category_num',
                        'male',
                        'marital_status_married',
                        'marital_status_single',
                        'marital_status_divorced',
                        "months_on_book",
                        "credit_limit",
                        "total_revolving_bal",
                        "avg_open_to_buy","total_trans_amt"]


f2_scorer = make_scorer(fbeta_score, beta=2)

X = data[selected_feature_names]
y = data[Dependent_var]

# Assuming X and y are your original feature matrix and target variable
# Use oversampling to address class imbalance
os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)
OVERSAMPLE_X, OVERSAMPLE_y = os.fit_resample(X_train, y_train)

xgb_model = XGBClassifier()
param_grid_xgboost = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

xgb_random_search = RandomizedSearchCV(
    xgb_model,
    param_distributions=param_grid_xgboost,
    n_iter=200,
    scoring=f2_scorer,
    cv=15,
    random_state=42,
    n_jobs=-1
)

# Fit the RandomizedSearchCV
xgb_random_search.fit(OVERSAMPLE_X, OVERSAMPLE_y)

# Get the best parameters and the best estimator
best_params_xgb = xgb_random_search.best_params_

# Print the best parameters
print("Best Parameters for XGBoost:")
print(best_params_xgb)

best_estimator_xgb = xgb_random_search.best_estimator_

explainer = shap.Explainer(best_estimator_xgb)
shap_values = explainer(X)



KeyError: "None of [Index(['customer_age', 'dependent_count', 'total_relationship_count',\n       'months_inactive_12_mon', 'contacts_count_12_mon',\n       'total_amt_chng_q4_q1', 'total_trans_ct', 'total_ct_chng_q4_q1',\n       'avg_utilization_ratio', 'education_level_num', 'income_category_num',\n       'card_category_num', 'male', 'marital_status_married',\n       'marital_status_single', 'marital_status_divorced', 'months_on_book',\n       'credit_limit', 'total_revolving_bal', 'avg_open_to_buy',\n       'total_trans_amt'],\n      dtype='object')] are in the [columns]"

In [ ]:
shap.plots.waterfall(shap_values[0])


In [ ]:
shap.plots.initjs()
shap.plots.force(shap_values[:500])

In [ ]:
shap.plots.beeswarm(shap_values)

In [ ]:
shap.plots.bar(shap_values)

In [ ]:




y_pred = best_estimator_xgb.predict(X_test)

shap.plots.beeswarm(shap_values)

# Calculate the F2 score on the test set
fbeta_xgb = fbeta_score(y_test, y_pred, beta=2)

# Print the results
print("XGBoost Model:")
print(f"F2 Score on Test Set: {fbeta_xgb:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
feature_labels = {
    'customer_age': 'Age',
    'dependent_count': 'Dependent Count',
    'total_relationship_count': 'Total Relationship Count',
    'months_inactive_12_mon': 'Months Inactive (Last 12 Months)',
    'contacts_count_12_mon': 'Contacts Count (Last 12 Months)',
    'total_amt_chng_q4_q1': 'Total Amount Change (Q4 to Q1)',
    'total_trans_ct': 'Total Transaction Count',
    'total_ct_chng_q4_q1': 'Total Count Change (Q4 to Q1)',
    'avg_utilization_ratio': 'Average Utilization Ratio',
    'education_level_num': 'Education Level Numeric',
    'income_category_num': 'Income Category Numeric',
    'card_category_num': 'Card Category Numeric',
    'male': 'Is Male',
    'marital_status_married': 'Marital Status: Married',
    'marital_status_single': 'Marital Status: Single',
    'marital_status_divorced': 'Marital Status: Divorced',
    "months_on_book": 'Months on Book',
    "credit_limit": 'Credit Limit',
    "total_revolving_bal": 'Total Revolving Balance',
    "avg_open_to_buy": 'Average Open to Buy',
    "total_trans_amt": 'Total Transaction Amount'
}


from shapash import SmartExplainer

# Create a SmartExplainer instance
xpl = SmartExplainer(
    model=best_estimator_xgb,
    features_dict=feature_labels
)

# Compile the explainer
xpl.compile(
    x=X_test,  # Assuming X_test is a DataFrame, and you want to explain the first row
    y_target=y_test # Optional: allows to display True Values vs Predicted Values

)

xpl.save('Output/Dashboard/ShapDashboardInterpretation.pkl')